In [1]:
import os
from estnltk import Text
from estnltk_neural.taggers import StanzaSyntaxTagger
import pandas as pd
import numpy as np
import sklearn

In [2]:
atomic_phrases = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases.pkl')

In [3]:
to_be_kept = [True]*len(atomic_phrases)
for idx, row in atomic_phrases.iterrows():
    if row['phrase_length'] < 2:
        to_be_kept[idx] = False

In [4]:
atomic_phrases_filtered = atomic_phrases[to_be_kept]

In [5]:
atomic_phrases_filtered

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,...,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex,pos_sequence_verb_info,graph_verb_info,graph_code_verb_info,graph_code_pos_verb_info,graph_code_pos_ner_timex_verb_info
1,Text(text='libedavastast graniidipuru'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(22, 48)",0,...,A-S,"(1, 2, 0)","((0, 2, root),(2, 1, amod))","(S-A,(0, 2, root),(2, 1, amod))","(0-0,S-A,(0, 2, root),(2, 1, amod))",A-S,"(1, 2, 0)","((0, 2, root),(2, 1, amod))","(S-A,(0, 2, root),(2, 1, amod))","(0-0,S-A,(0, 2, root),(2, 1, amod))"
2,Text(text='juba vallid'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(114, 125)",0,...,D-S,"(1, 2, 0)","((0, 2, root),(2, 1, advmod))","(S-D,(0, 2, root),(2, 1, advmod))","(0-0,S-D,(0, 2, root),(2, 1, advmod))",D-S,"(1, 2, 0)","((0, 2, root),(2, 1, advmod))","(S-D,(0, 2, root),(2, 1, advmod))","(0-0,S-D,(0, 2, root),(2, 1, advmod))"
6,Text(text='neist teede'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nmod_phrase,"(84, 95)",0,...,P-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-P,(0, 2, root),(2, 1, nmod))","(0-0,S-P,(0, 2, root),(2, 1, nmod))",P-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-P,(0, 2, root),(2, 1, nmod))","(0-0,S-P,(0, 2, root),(2, 1, nmod))"
9,Text(text='Defender-18 : Sokrates korras sellega ?'),6,2024-03-10T00:43,20676731,705250,"(65236, 65274)",jututoavestlus,root_phrase,"(0, 38)",0,...,N-Z-H-S-P-Z,"(1, 4, 2, 3, 0, 5, 6)","((0, 4, root),(4, 1, parataxis),(4, 2, punct),(4, 3, nmod),(4, 5, nmod),(4, 6, punct))","(S-N-Z-H-P-Z,(0, 4, root),(4, 1, parataxis),(4, 2, punct),(4, 3, nmod),(4, 5, nmod),(4, 6, punct))","(0-0-0-0-0-0,S-N-Z-H-P-Z,(0, 4, root),(4, 1, parataxis),(4, 2, punct),(4, 3, nmod),(4, 5, nmod),(4, 6, punct))",N-Z-H-S-P-Z,"(1, 4, 2, 3, 0, 5, 6)","((0, 4, root),(4, 1, parataxis),(4, 2, punct),(4, 3, nmod),(4, 5, nmod),(4, 6, punct))","(S-N-Z-H-P-Z,(0, 4, root),(4, 1, parataxis),(4, 2, punct),(4, 3, nmod),(4, 5, nmod),(4, 6, punct))","(0-0-0-0-0-0,S-N-Z-H-P-Z,(0, 4, root),(4, 1, parataxis),(4, 2, punct),(4, 3, nmod),(4, 5, nmod),(4, 6, punct))"
12,Text(text='Tartu ülikooli'),2,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 14)",0,...,H-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-H,(0, 2, root),(2, 1, nmod))","(0-0,S-H,(0, 2, root),(2, 1, nmod))",H-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-H,(0, 2, root),(2, 1, nmod))","(0-0,S-H,(0, 2, root),(2, 1, nmod))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36052,Text(text='Pekingi slummis'),2,2024-03-10T01:35,8262548,396490,"(5476, 5574)",aja_ee,obl_phrase,"(6, 21)",0,...,H-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-H,(0, 2, root),(2, 1, nmod))","(0-0,S-H,(0, 2, root),(2, 1, nmod))",H-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-H,(0, 2, root),(2, 1, nmod))","(0-0,S-H,(0, 2, root),(2, 1, nmod))"
36053,Text(text='oma hurtsikus'),2,2024-03-10T01:35,8262548,396490,"(5476, 5574)",aja_ee,obl_phrase,"(51, 64)",0,...,P-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-P,(0, 2, root),(2, 1, nmod))","(0-0,S-P,(0, 2, root),(2, 1, nmod))",P-S,"(1, 2, 0)","((0, 2, root),(2, 1, nmod))","(S-P,(0, 2, root),(2, 1, nmod))","(0-0,S-P,(0, 2, root),(2, 1, nmod))"
36056,Text(text='mingit sööki'),2,2024-03-10T01:35,8262548,396490,"(5476, 5574)",aja_ee,obj_phrase,"(71, 83)",0,...,P-S,"(1, 2, 0)","((0, 2, root),(2, 1, det))","(S-P,(0, 2, root),(2, 1, det))","(0-0,S-P,(0, 2, root),(2, 1, det))",P-S,"(1, 2, 0)","((0, 2, root),(2, 1, det))","(S-P,(0, 2, root),(2, 1, det))","(0-0,S-P,(0, 2, root),(2, 1, det))"
36059,Text(text='Laulupeo eelproovide käigus'),3,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obl_phrase,"(0, 27)",0,...,S-S-K,"(1, 2, 0, 3)","((0, 2, root),(2, 1, nmod),(2, 3, case))","(S-S-K,(0, 2, root),(2, 1, nmod),(2, 3, case))","(0-0-0,S-S-K,(0, 2, root),(2, 1, nm

In [6]:
grouped_phrases = atomic_phrases_filtered.groupby('graph_code_verb_info')

In [7]:
grouped_phrases.ngroups

2048

In [8]:
sorted_groups = grouped_phrases.size().reset_index(name='count').sort_values(['count'], ascending=False)

In [9]:
# top50 syntax trees
top50_trees = dict()
idx = 0
for i, row in sorted_groups.iterrows():
    top50_trees[row['graph_code_verb_info']] = row['count']
    idx+=1
    if idx == 50:
        break

In [10]:
total = 0
for el in top50_trees:
    print(el, top50_trees[el])
    total+=(top50_trees[el])
    
print(total)

((0, 2, root),(2, 1, nmod)) 3364
((0, 2, root),(2, 1, amod)) 2111
((0, 1, root),(1, 2, flat)) 1034
((0, 2, root),(2, 1, det)) 944
((0, 1, root),(1, 2, case)) 757
((0, 2, root),(2, 1, nummod)) 569
((0, 2, root),(2, 1, advmod)) 418
((0, 1, root),(1, 2, nmod)) 290
((0, 2, root),(2, 1, acl)) 246
((0, 3, root),(3, 1, nmod),(3, 2, amod)) 238
((0, 2, root),(2, 1, case)) 190
((0, 2, root),(2, 1, nmod),(2, 3, case)) 187
((0, 1, root),(1, 2, nummod)) 122
((0, 2, root),(2, 1, punct),(2, 3, punct)) 120
((0, 3, root),(2, 1, advmod),(3, 2, amod)) 111
((0, 1, root),(1, 2, punct),(1, 3, parataxis)) 110
((0, 1, root),(1, 2, appos),(2, 3, flat)) 109
((0, 2, root),(2, 1, nmod),(2, 3, appos),(3, 4, flat)) 106
((0, 2, root),(2, 1, amod),(2, 3, case)) 100
((0, 2, root),(2, 1, nummod),(2, 3, case)) 88
((0, 3, root),(3, 1, det),(3, 2, amod)) 87
((0, 1, root),(1, 2, appos)) 75
((0, 2, root),(2, 1, punct)) 75
((0, 3, root),(2, 1, advmod),(3, 2, nummod)) 74
((0, 3, root),(3, 1, case),(3, 2, nmod)) 71
((0, 2, roo

In [19]:
# finding phrases that have ner/timex entity
df = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases_reverse_top50.pkl')

new_df2 = pd.DataFrame()

for idx, row in df.iterrows():
    if 'ner' in row['graph_code_pos_ner_timex_verb_info'] or 'timex' in row['graph_code_pos_ner_timex_verb_info']:
        temp_data = {}
        temp_data.update(row)
        temp = pd.DataFrame.from_records([temp_data])
        new_df2 = pd.concat([new_df2, temp], ignore_index=True)

In [20]:
grouped_df2 = new_df2.groupby('graph_code_verb_info')

In [21]:
grouped_df2.ngroups

40

In [22]:
# finding trees based on those phrases
trees = []

for name, group in grouped_df2:
    trees.append(name)

In [23]:
trees

['((0, 1, root),(1, 2, appos),(2, 3, flat))',
 '((0, 1, root),(1, 2, case))',
 '((0, 1, root),(1, 2, flat))',
 '((0, 1, root),(1, 2, flat),(1, 3, flat))',
 '((0, 1, root),(1, 2, nmod))',
 '((0, 1, root),(1, 2, nummod))',
 '((0, 1, root),(1, 2, punct),(1, 3, nmod))',
 '((0, 1, root),(1, 2, punct),(1, 3, parataxis))',
 '((0, 1, root),(1, 3, parataxis),(3, 2, punct))',
 '((0, 2, root),(2, 1, acl))',
 '((0, 2, root),(2, 1, advmod))',
 '((0, 2, root),(2, 1, advmod),(2, 3, case))',
 '((0, 2, root),(2, 1, amod))',
 '((0, 2, root),(2, 1, amod),(2, 3, case))',
 '((0, 2, root),(2, 1, amod),(2, 3, nmod))',
 '((0, 2, root),(2, 1, case))',
 '((0, 2, root),(2, 1, det))',
 '((0, 2, root),(2, 1, det),(2, 3, case))',
 '((0, 2, root),(2, 1, nmod))',
 '((0, 2, root),(2, 1, nmod),(2, 3, appos),(3, 4, flat))',
 '((0, 2, root),(2, 1, nmod),(2, 3, case))',
 '((0, 2, root),(2, 1, nmod),(2, 3, nmod))',
 '((0, 2, root),(2, 1, nummod))',
 '((0, 2, root),(2, 1, parataxis))',
 '((0, 2, root),(2, 1, punct),(2, 3, p

In [16]:
def tree_to_markdown(text_obj):
    markdown = ''
    for i in range(len(text_obj.stanza_syntax)):
        stanza_word = text_obj.stanza_syntax[i]
        ner = None
        nertag = None
        
        if len(text_obj.ner) > 0:
            word = text_obj.words.get(stanza_word)
            for n in text_obj.ner:
                for part in n:
                    if part==word:
                        ner=word
                        nertag=n.nertag
                        
        timex = text_obj.timexes.get(stanza_word)
        if ner:
            markdown += f"{stanza_word.id}\t{nertag}\t{stanza_word.text}\t{stanza_word.head}\t{stanza_word.deprel}\n\n"
        elif timex:
            markdown += f"{stanza_word.id}\tTIMEX\t{stanza_word.text}\t{stanza_word.head}\t{stanza_word.deprel}\n\n"
        else:
            markdown += f"{stanza_word.id}\tOTHER\t{stanza_word.text}\t{stanza_word.head}\t{stanza_word.deprel}\n\n"
    return markdown

In [29]:
def df_to_markdown(df):
    markdown = ''
    for _, row in df.iterrows():
        markdown += f"{row['pos_sequence']}: {row['%_pos_sequence']}%\n\nExample tree:\n{row['example_tree']}Count of tree: {row['count_of_tree']}\n\n% of all trees: {row['%_all_trees']}\n\n% of ORG: {row['%_ORG']}\n\n% of LOC: {row['%_LOC']}\n\n% of PER: {row['%_PER']}\n\n\n"
    return markdown

In [24]:
df = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases_reverse_top50.pkl')

In [30]:
# phrases with ner
grouped_df_top50 = df.groupby('graph_code_verb_info')

n_phrases = len(df)

md_content = ""

# creating data
#data = {'tree_graph': [], 'example_tree': [], 'pos_sequence': [], '%_phrases_with_ner': [],  'count_of_tree': [], '%_all_trees': []}

for name, group in grouped_df_top50:
    # creating data
    data = {'pos_sequence': [], '%_pos_sequence': [], 'example_tree': [], 'count_of_tree': [], '%_all_trees': [], '%_ORG': [], '%_LOC': [], '%_PER': []}

    if name in trees:
        grouped_2 = group.groupby('pos_sequence_verb_info')
        for name2, group2 in grouped_2:
            group2_len = len(group2)
            ner_ORG = 0
            ner_LOC = 0
            ner_PER = 0
            for idx, row in group2.iterrows():
                if 'ORG' in row['graph_code_pos_ner_timex_verb_info']:
                    ner_ORG+=1
                elif 'LOC' in row['graph_code_pos_ner_timex_verb_info']:
                    ner_LOC+=1
                elif 'PER' in row['graph_code_pos_ner_timex_verb_info']:
                    ner_PER+=1
            if ner_ORG > 0 or ner_LOC > 0 or ner_PER > 0:
                # example will be chosen randomly from phrases with NER entity
                group2 = sklearn.utils.shuffle(group2).reset_index(drop=True)
                example = None
                for idx, row in group2.iterrows():
                    if row['has_ner_netity'] == 1:
                        example = row
                        break
                data['pos_sequence'].append(name2)
                data['%_pos_sequence'].append(round((len(group2)/len(group))*100, 2))
                data['example_tree'].append(tree_to_markdown(example['phrase']))
                data['count_of_tree'].append(len(group2))
                data['%_all_trees'].append(round((len(group2)/n_phrases)*100, 2))
                data['%_ORG'].append((ner_ORG/group2_len)*100)
                data['%_LOC'].append((ner_LOC/group2_len)*100)
                data['%_PER'].append((ner_PER/group2_len)*100)
                
        
        data_df = pd.DataFrame(data)
        md_content+=f"## {name}\n\n"
        md_content+=df_to_markdown(data_df)  
print(md_content)
#with open('phrase_examples_filtered_sub10000/ner_trees_top50_atomic_new.md', 'w') as f:
#    f.write(md_content)

## ((0, 1, root),(1, 2, appos),(2, 3, flat))

H-H-H: 3.67%

Example tree:
1	LOC	Rootsi	0	root

2	OTHER	Toomas	1	appos

3	OTHER	Tiiveli	2	flat

Count of tree: 4

% of all trees: 0.03

% of ORG: 0.0

% of LOC: 50.0

% of PER: 0.0


S-H-S: 1.83%

Example tree:
1	OTHER	kütusefirma	0	root

2	ORG	Olerex	1	appos

3	OTHER	turundusjuhina	2	flat

Count of tree: 2

% of all trees: 0.02

% of ORG: 50.0

% of LOC: 0.0

% of PER: 0.0


## ((0, 1, root),(1, 2, case))

H-K: 6.08%

Example tree:
1	PER	Arturi	0	root

2	OTHER	juurde	1	case

Count of tree: 46

% of all trees: 0.37

% of ORG: 2.1739130434782608

% of LOC: 50.0

% of PER: 6.521739130434782


S-K: 63.01%

Example tree:
1	ORG	Riigikogu	0	root

2	OTHER	ette	1	case

Count of tree: 477

% of all trees: 3.79

% of ORG: 0.628930817610063

% of LOC: 0.0

% of PER: 0.0


Y-K: 1.19%

Example tree:
1	ORG	KGB	0	root

2	OTHER	heaks	1	case

Count of tree: 9

% of all trees: 0.07

% of ORG: 22.22222222222222

% of LOC: 11.11111111111111

% of PER: 0.0


#